In [1]:
import os
from dotenv import load_dotenv
from nnsight import CONFIG, LanguageModel

load_dotenv(override=True)


CONFIG.API.APIKEY = os.getenv("NDIF_KEY")
# model_id, remote, device = "meta-llama/Llama-3.1-70B", True, "auto"
# model_id, remote, device = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B", True, "auto"
model_id, remote, device = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B", True, "auto"
# model_id, remote, device = "HuggingFaceTB/SmolLM2-135M-Instruct", False, "mps"
model = LanguageModel(model_id, device_map=device, trust_remote_code=True)

/Users/aghyaddeeb/Documents/coding/reasoning_or_performing/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_special_tokens():
    # temp = [{"role": "assistant", "content": ""}, {"role": "user", "content": ""}, {"role"}]
    # assistant = model.tokenizer.apply_chat_template(temp, tokenize=False, add_generation_prompt=True)
    # print(assistant)
    think = model.tokenizer("<think>", add_special_tokens=False)["input_ids"]
    end_of_think = model.tokenizer("</think>", add_special_tokens=False)["input_ids"]
    return dict(think=think, end_of_think=end_of_think)

get_special_tokens()

{'think': [128013], 'end_of_think': [128014]}

In [3]:
# think_token, end_of_think_token = get_special_tokens()
think_token = get_special_tokens()["think"]
end_of_think_token = get_special_tokens()["end_of_think"]
# print(end_of_think_token)
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids, scores, **kwargs):
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=[end_of_think_token])])

with model.generate("hi", remote=remote, stopping_criteria=stopping_criteria, max_new_tokens=1000):
# with model.generate("hi", remote=remote, stopping_criteria=stopping_criteria):
# with model.generate("hi", remote=remote, stopping_criteria=stopping_criteria):
# with model.generate("hi", remote=remote, max_new_tokens=1000):
prompt = "hi"
temp = [
    {"role": "user", "content": prompt},
]
input = model.tokenizer.apply_chat_template(temp, tokenize=True, add_generation_prompt=True)
with model.generate(input, remote=remote, eos_token_id=end_of_think_token, max_new_tokens=1000):
    out =  model.generator.output.save()
model.tokenizer.decode(out[0])
# out
# out[0]

IndentationError: expected an indented block after 'with' statement on line 18 (2169851429.py, line 22)

In [12]:
model.tokenizer.decode([0])

'!'

In [14]:
import torch
def find_first_token_index(tensor, value) -> int:
    """Find the first index where tensor equals value."""
    if isinstance(value, list):
        value = torch.tensor(value)
    if not isinstance(tensor, torch.Tensor):
        tensor = torch.tensor(tensor).view((-1))
    print(f"{tensor=}{value=}")
    indices = (tensor == value).nonzero()

    if len(indices) > 0:
        return indices[0].item()
    return None

def find_last_token_index(tensor, value) -> int:
    # assert isinstance(tensor, torch.Tensor), f"{tensor=}"
    if isinstance(value, list):
        value = torch.tensor(value)
    if not isinstance(tensor, torch.Tensor):
        tensor = torch.tensor(tensor).view((-1))
    print(f"{tensor=}{value=}")
    indices = (tensor == value).nonzero()
    # print(f"{indices=}")
    if len(indices) > 0:
        return indices[-1].item()
    return None

def find_length_between_tokens(tensor, start_token, end_token) -> int:
    end = find_last_token_index(tensor, end_token)
    start = find_first_token_index(tensor, start_token)
    print(f"{end=} {start=}")
    return end - start
    
def get_logits(prompt):
    print(f"prompt from get_logits_after_thinking: {prompt=}")
    temp = [
        {"role": "user", "content": prompt},
    ]
    input = model.tokenizer.apply_chat_template(temp, tokenize=True, add_generation_prompt=True)
    print(f"{model.tokenizer.decode(input)=}")
    with model.trace(input, remote=remote):
        out = model.output.save()
    logits = out["logits"][0]
    return logits

def generate_till_eos(prompt, max_new_tokens=1000):
    temp = [
        {"role": "user", "content": prompt},
    ]
    input = model.tokenizer.apply_chat_template(temp, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    # input = model.tokenizer.apply_chat_template(temp, tokenize=False, add_generation_prompt=True)
    # print(f"{model.tokenizer.decode(input)=}")
    # display(f"{input=}")
    # display(f"{input.shape=}")
    # while True:
    #     with model.trace(input, remote=remote):
    #         out = model.output.save()
    #     display(f"{out.shape=}")
    #     logits = out["logits"][0, -1]
    #     new_token = logits.argmax()
    #     print(f"{new_token.shape}")
    #     print(f"{torch.unsqueeze(new_token, 0).shape}")
    #     input = torch.cat((input, torch.unsqueeze(torch.unsqueeze(new_token, 0), 0)), dim=-1)
    #     if new_token == end_of_think:
    #         break
    with model.generate(input, remote=remote, max_new_tokens=max_new_tokens):
    # with model.generate(input, remote=remote, max_new_tokens=max_new_tokens):
        prompt_with_thinking = model.generator.output.save()

    return prompt_with_thinking

def get_logits_after_thinking(prompt, max_new_tokens=1000, get_full_generation=False):
    special_tokens = get_special_tokens()
    think_token = special_tokens["think"]
    end_of_think_token = special_tokens["end_of_think"]
    print(f"prompt from get_logits: {prompt=}")
    temp = [
        {"role": "user", "content": prompt},
    ]
    input = model.tokenizer.apply_chat_template(temp, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    # input = model.tokenizer.apply_chat_template(temp, tokenize=False, add_generation_prompt=True)
    # print(f"{model.tokenizer.decode(input)=}")
    # display(f"{input=}")
    # display(f"{input.shape=}")
    # while True:
    #     with model.trace(input, remote=remote):
    #         out = model.output.save()
    #     display(f"{out.shape=}")
    #     logits = out["logits"][0, -1]
    #     new_token = logits.argmax()
    #     print(f"{new_token.shape}")
    #     print(f"{torch.unsqueeze(new_token, 0).shape}")
    #     input = torch.cat((input, torch.unsqueeze(torch.unsqueeze(new_token, 0), 0)), dim=-1)
    #     if new_token == end_of_think:
    #         break
    with model.generate(input, remote=remote, eos_token_id=end_of_think_token, max_new_tokens=max_new_tokens):
    # with model.generate(input, remote=remote, max_new_tokens=max_new_tokens):
        prompt_with_thinking = model.generator.output.save()

    thinking_length = find_length_between_tokens(
        prompt_with_thinking, think_token, end_of_think_token
    )
    

    wanted_tokens = [32, 33, 34, 35]
    def get_one():
        with model.trace(prompt_with_thinking, remote=remote):
            logits = model.output[0][-1][-1][wanted_tokens].save()
        return logits
    # Get the logits for the specified tokens
    logits = get_one()
    
    # Get the token with the maximum probability
    max_prob_token_idx = logits.argmax(dim=-1).item()
    while max_prob_token_idx not in wanted_tokens:
        print(f"{max_prob_token_idx=}"")
        print(f"{prompt_with_thinking.shape=}")
        print(f"{logits=}")
        prompt_with_thinking = torch.cat(prompt_with_thinking, )
        logits = get_one()
        max_prob_token_idx = logits.argmax().item()

    full = []
    if get_full_generation:
        with model.generate(prompt_with_thinking, remote=remote, max_new_tokens=max_new_tokens):
        # with model.generate(input, remote=remote, max_new_tokens=max_new_tokens):
            full = model.generator.output.save()

    # Convert to a tensor with the same dtype as the original logits
    logits = logits.to(logits.dtype)

    display(f"{logits.shape=}")
    return logits, prompt_with_thinking, thinking_length, full

SyntaxError: unterminated string literal (detected at line 116) (3755689052.py, line 116)

In [11]:
prompt = """Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A) 0
B) 4
C) 2
D) 6
After thinking about the question, output only one of the letters A, B, C, D. There should be nothing between the </think> token and the letter you output"""
output = get_logits_after_thinking(prompt=prompt, max_new_tokens=9999)
model.tokenizer.decode(output[1].view((-1)))
model.tokenizer.decode(output[0].view((-1)).argmax())

prompt from get_logits: prompt='Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.\nA) 0\nB) 4\nC) 2\nD) 6\nAfter thinking about the question, output only one of the letters A, B, C, D. There should be nothing between the </think> token and the letter you output'


2025-03-27 16:05:55,598 3b35a50a-d8b6-4976-806e-088d028f31cf - RECEIVED: Your job has been received and is waiting approval.
2025-03-27 16:05:56,110 3b35a50a-d8b6-4976-806e-088d028f31cf - APPROVED: Your job was approved and is waiting to be run.
2025-03-27 16:05:56,623 3b35a50a-d8b6-4976-806e-088d028f31cf - RUNNING: Your job has started running.
2025-03-27 16:06:22,124 3b35a50a-d8b6-4976-806e-088d028f31cf - COMPLETED: Your job has been completed.


tensor=tensor([128000, 128011,  16533,   1193,    449,    362,     11,    426,     11,
           356,    477,    423,     13,  16225,     25,   7531,    279,   8547,
           369,    279,   2728,   2115,   9070,   1229,  84173,      7,     17,
           705,  18430,      7,     18,    705,  18430,      7,    972,    595,
           927,   1229,    627,     32,      8,    220,     15,    198,     33,
             8,    220,     19,    198,     34,      8,    220,     17,    198,
            35,      8,    220,     21,    198,   6153,   7422,    922,    279,
          3488,     11,   2612,   1193,    832,    315,    279,  12197,    362,
            11,    426,     11,    356,     11,    423,     13,   2684,   1288,
           387,   4400,   1990,    279,    220, 128014,   4037,    323,    279,
          6661,    499,   2612, 128012, 128013,    198,  72586,     11,    779,
           358,    617,    420,   3488,   1618,     25,   7531,    279,   8547,
           315,    279,   2115,  

2025-03-27 16:06:25,921 ae4bbb38-5016-49e2-aac7-5472e9cc19b6 - RECEIVED: Your job has been received and is waiting approval.
2025-03-27 16:06:26,727 ae4bbb38-5016-49e2-aac7-5472e9cc19b6 - APPROVED: Your job was approved and is waiting to be run.
2025-03-27 16:06:28,031 ae4bbb38-5016-49e2-aac7-5472e9cc19b6 - RUNNING: Your job has started running.
2025-03-27 16:06:30,107 ae4bbb38-5016-49e2-aac7-5472e9cc19b6 - COMPLETED: Your job has been completed.


0


...

0


2025-03-27 16:06:51,507 9e21bfca-1dc8-4814-88fb-b65b4bbd370e - RECEIVED: Your job has been received and is waiting approval.
2025-03-27 16:06:52,021 9e21bfca-1dc8-4814-88fb-b65b4bbd370e - APPROVED: Your job was approved and is waiting to be run.
2025-03-27 16:06:52,327 9e21bfca-1dc8-4814-88fb-b65b4bbd370e - RUNNING: Your job has started running.
2025-03-27 16:06:52,654 9e21bfca-1dc8-4814-88fb-b65b4bbd370e - COMPLETED: Your job has been completed.


0


2025-03-27 16:06:53,556 75e69b6a-4fc4-4828-9515-024e17d419f6 - RECEIVED: Your job has been received and is waiting approval.
2025-03-27 16:06:53,863 75e69b6a-4fc4-4828-9515-024e17d419f6 - APPROVED: Your job was approved and is waiting to be run.


KeyboardInterrupt: 

In [8]:
model.tokenizer("A"), model.tokenizer("B"), model.tokenizer("C"), model.tokenizer("D"),
["A", "B", "C", "D"][output[0].softmax(-1).argmax()]
# add a short sentence "the answer is "

'A'

In [112]:
output[0].softmax(-1)

tensor([0.5586, 0.1289, 0.1260, 0.1855], dtype=torch.bfloat16)

In [96]:
# model.tokenizer.decode(output[0].argmax())
def get_answer_probabilities(logits, choices_count):
    import torch.nn.functional as F
    # Get probabilities for the first token after the prompt
    probs = F.softmax(logits, dim=-1)
    
    # Map to token IDs for A, B, C, D, E
    answer_tokens = [model.tokenizer.encode(chr(65+i))[0] for i in range(choices_count)]
    
    # Get probabilities for these tokens
    answer_probs = [probs[token_id].item() for token_id in answer_tokens]
    return answer_probs

get_answer_probabilities(output[0], 4)

[1.0, 1.0, 1.0, 1.0]

In [ ]:
from datasets import load_dataset
import torch
import torch.nn.functional as F

dataset_name = "cais/mmlu"
#! Use subjectyou know to understand reasoning
dataset = load_dataset(dataset_name, "abstract_algebra", split="test")  # Let's start with just 10 examples

def format_mcq(question, choices):
    # Format the question with choices as A, B, C, D, E
    prompt = f"Answer only with A, B, C or D. Question: {question}\n"
    for i, choice in enumerate(choices):
        prompt += f"{chr(65+i)}) {choice}\n"
    prompt += "Answer: "
    display(prompt)
    return prompt

def get_answer_probabilities(logits, choices_count):
    # logits should be of dimensions (num_tokens)
    # Get probabilities for the first token after the prompt
    probs = F.softmax(logits, dim=-1)
    
    # Map to token IDs for A, B, C, D, E
    answer_tokens = [model.tokenizer.encode(chr(65+i))[0] for i in range(choices_count)]
    
    # Get probabilities for these tokens
    answer_probs = [probs[token_id].item() for token_id in answer_tokens]
    return answer_probs

# Test the model on some examples
correct = 0
cnt = 0
log = []
for example in dataset:
    if cnt >= 5:
        break
    cnt += 1
    # display(example)
    # assert False
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']  # A, B, C, D, or E
    
    # Format the question
    prompt = format_mcq(question, choices)
    
    # Get model's logits
    output = get_logits_after_thinking(prompt, max_new_tokens=9999)
    logits = output[0]
    log.append(dict(example=example, output=output))
    
    # Get probabilities for each answer choice
    probs = get_answer_probabilities(logits, len(choices))
    
    # Get model's prediction
    pred_idx = max(range(len(probs)), key=lambda i: probs[i])
    pred_answer = chr(65 + pred_idx)
    
    # Print results
    print(f"\nQuestion: {question}")
    print("Choices:")
    for i, choice in enumerate(choices):
        print(f"{chr(65+i)}) {choice}")
    print(f"Correct answer: {correct_answer}")
    print(f"Model predicted: {pred_answer}")
    print(f"{pred_idx=}")
    print(f"{pred_idx==correct_answer=}")
    print(f"Answer probabilities: {dict(zip('ABCDE', probs))}")
    
    if pred_answer == correct_answer:
        correct += 1

print(f"\nAccuracy: {correct/len(dataset)*100:.2f}%")
#! Need to get reasoning length and find longest answers.

'Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.\nA) 0\nB) 4\nC) 2\nD) 6\nAnswer: '

Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A) 0
B) 4
C) 2
D) 6
Answer: 
prompt from get_logits: prompt='Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.\nA) 0\nB) 4\nC) 2\nD) 6\nAnswer: '
model.tokenizer.decode(input)='<｜begin▁of▁sentence｜><｜User｜>Answer only with A, B, C or D. Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.\nA) 0\nB) 4\nC) 2\nD) 6\nAnswer: <｜Assistant｜><think>\n'


KeyboardInterrupt: 

In [21]:
# remote = True means the model will execute on NDIF's shared resources
with model.trace("The Eiffel Tower is in the city of", remote=True):

    # user-defined code to access internal model components
    model.model.layers[7].output[0][:] = 4 # in-place operation to change a single layer's output values
    output = model.output.save()

# after exiting the tracing context, we can access any values that were saved

output_logits = output["logits"]
print("Model Output Logits: ",output_logits[0])

# decode the final model output from output logits
max_probs, tokens = output_logits[0].max(dim=-1)
word = [model.tokenizer.decode(tokens.cpu()[-1])]
print("Model Output: ", word[0])

2025-03-10 22:52:04,933 98de260a-f4b5-45f2-b23e-9dabefb2be5f - RECEIVED: Your job has been received and is waiting approval.
2025-03-10 22:52:05,391 98de260a-f4b5-45f2-b23e-9dabefb2be5f - APPROVED: Your job was approved and is waiting to be run.
2025-03-10 22:52:05,576 98de260a-f4b5-45f2-b23e-9dabefb2be5f - RUNNING: Your job has started running.
2025-03-10 22:52:07,226 98de260a-f4b5-45f2-b23e-9dabefb2be5f - COMPLETED: Your job has been completed.

Model Output Logits:  tensor([[-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086],
        [-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086],
        [-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086],
        ...,
        [-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086],
        [-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086],
        [-1.4766,  0.9492, -0.9688,  ..., -0.3086, -0.3086, -0.3086]],
       dtype=torch.bfloat16)
Model Output:   Bounty
